# Assignment 9

## Explanation of each step

Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Importing external files

In [4]:
ExternalFilesFolder = r"C:\Users\Famiglia\Documents\Manuel\Polimi\MAGISTRALE\PRIMO ANNO\Primo semestre\Bezhad\python4ScientificComputing_Numpy_Pandas_MATPLotLIB\ExternalFiles"
ConsumptionFileName= "consumption_5545.csv"
TemperatureFileName= "Austin_weather_2014.csv"
IrradianceFileName= "irradiance_2014_gen.csv"

path_consumptionFile = os.path.join(ExternalFilesFolder,ConsumptionFileName)
path_TemperatureFile = os.path.join(ExternalFilesFolder,TemperatureFileName)
path_IrradianceFile = os.path.join(ExternalFilesFolder,IrradianceFileName)

DF_consumption = pd.read_csv(path_consumptionFile,sep=",", index_col=0)

Converting array index into a date time index with pd.to_datetime

In [ ]:
PreviousIndex = DF_consumption.index
NewParsedIndex= pd.to_datetime(PreviousIndex)
DF_consumption.index =NewParsedIndex 

Choosing a selected period of time to work on and to plot it in two ways

In [5]:
DF_consumption_someDaysInJuly=DF_consumption["2014-07-01 00:00:00":"2014-07-03 23:00:00"]

plt.figure()
plt.plot(DF_consumption_someDaysInJuly)
plt.xlabel("Time")
plt.ylabel("AC Power (W)")
plt.show()

# There is asecond way of doing this !!!
plt.figure()
DF_consumption_someDaysInJuly.plot()
plt.xlabel("Time")
plt.ylabel("AC Power (W)")
plt.show()

ValueError: invalid literal for float(): 2014-07-03 23:00:00

Importing the weather data, changing the index into a date time index and creating a series of temperatures

In [11]:
DF_weather = pd.read_csv(path_TemperatureFile,sep=";",index_col=0)

previousIndex_weather=DF_weather.index
newIndex_weather=pd.to_datetime(previousIndex_weather)
DF_weather.index = newIndex_weather

DF_weather.columns
Series_Temperature = DF_weather["temperature"]

DF_Temperature= DF_weather[["temperature"]]

Importing irrandiance data, put them in a column and give value zero to all negative irradiance values

In [12]:
DF_irradianceSource = pd.read_csv(path_IrradianceFile,sep=";",index_col=1)

DF_irradiance=DF_irradianceSource[["gen"]]

DF_irradiance[DF_irradiance["gen"]<0] = 0

C:\Users\Famiglia\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Creating a matrix joining temperature and irradiance columns to the consumption column, basing all to the date time index

In [13]:
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])

Deleting all rows with a Nan included in consumption (empty spots), creating a copy with .copy to have cleaned one to work on, end step takes account on a specific period for calculating what's next

In [14]:
DF_joined_cleaned = DF_joined.dropna()
DF_joined_cleaned_copy = DF_joined.dropna().copy()

DF_joined_cleaned_chosenDates = DF_joined_cleaned_copy["2014-08-01":"2014-08-04"]

Shifting the temperature by the time zone lag of 5 hours and describing numerically what we have found

In [ ]:
DF_joined_cleaned_chosenDates["temperature"]=DF_joined_cleaned_chosenDates["temperature"].shift(-5)   #per il fuso orario
DF_joined_cleaned_chosenDates.dropna()
DF_joined_cleaned_chosenDates.head()
DF_joined_cleaned_chosenDates.describe()

Normalizing of data to have them in the same size graph and plotting them together

In [16]:
DF_joined_cleaned_chosenDates_min=DF_joined_cleaned_chosenDates.min()
DF_joined_cleaned_chosenDates_max=DF_joined_cleaned_chosenDates.max()
DF_joined_cleaned_chosenDates_normalized= (DF_joined_cleaned_chosenDates-DF_joined_cleaned_chosenDates_min)/(DF_joined_cleaned_chosenDates_max-DF_joined_cleaned_chosenDates_min)
plt.figure()
DF_joined_cleaned_chosenDates_normalized.plot()

Reconstructing the dataframe by applying time zone to the whole dataset

In [17]:
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])
DF_mod = DF_joined.copy()
DF_mod["temperature"]=DF_mod["temperature"].shift(-5)
DF_mod.dropna(inplace=True) # It is the same as writing it like this: DF_mod=DF_mod.dropna()

Better way to reconstruct the dataframe using a function that applies time zone shift to the dataset and returns different lags for studying the previous temperature values

In [18]:
""" Testing the wotking principle of the next function:
lag_start=1
lag_end = 6
lag_interval=1
 

column_name="temperature"
df=DF_mod
for i in range(lag_start,lag_end+1,lag_interval):
    new_column_name = column_name+" -"+str(i)+"hr"
    print new_column_name
    df[new_column_name]=df[column_name].shift(i)   
    df.dropna(inplace=True) #this removes all the row with a Nan
"""

def lag_feature(df,column_name,lag_start,lag_end,lag_interval):
    for i in range(lag_start,lag_end+1,lag_interval):
        new_column_name = column_name+" -"+str(i)+"hr"
        print new_column_name
        df[new_column_name]=df[column_name].shift(i)   
        df.dropna(inplace=True) #this removes all the row with a Nan
    return df

In [ ]:
Ways to rename column indexes

In [ ]:
DF_mod=DF_mod.rename(columns={"air conditioner_5545":"AC_consumption","gen":"irradiance"})

In [ ]:
Applying of the function to our three columns

In [19]:
#Previous 6 hours iterval temperature data 
DF_mod=  lag_feature(DF_mod,"temperature",1,6,1)

# Let's lag the irraidance , I would do it just for 3 to 6 hours
DF_mod=  lag_feature(DF_mod,"irradiance",3,6,1)

# Let's add the previous consumptions in the last 24 hours!
DF_mod=  lag_feature(DF_mod,"AC_consumption",1,24,1)
DF_mod.describe()

temperature -1hr
temperature -2hr
temperature -3hr
temperature -4hr
temperature -5hr
temperature -6hr
irradiance -3hr


KeyError: 'irradiance'

Adding the seasonality parameters that depends on time and checking correlation on cos-time and sin-time graph to our consumption-time graph

In [21]:
DF_mod["hour"]=DF_mod.index.hour
DF_mod["sin_hour"]=np.sin(DF_mod.index.hour*2*np.pi/24)
DF_mod["cos_hour"]=np.cos(DF_mod.index.hour*2*np.pi/24)

DF_mod["day_of_week"]=DF_mod.index.dayofweek
DF_mod["month"]=DF_mod.index.month
DF_mod["week_of_year"]=DF_mod.index.week

Creating two functions for selecting the weekend data and hours of the day that means when occupants are not at home, giving them as result a boolean column to then choose only value of flag variable equal to 1 for creating the columns "weekend" and "working time"

In [22]:
def WeekendDetector(day):
    if (day==5 or day == 6):
        weekendLabel=1
    else:
        weekendLabel=0
    return weekendLabel

DF_mod["weekend"]= DF_mod["day_of_week"].apply(WeekendDetector)
    
def DayDetector(hour):
    if (hour< 19 and hour>=9):
        DayLabel=1
    else:
        DayLabel=0
    return DayLabel

DF_mod["workingTime"] = DF_mod["hour"].apply(DayDetector)

Evaluate how much is the correlation of the consumption wrt all our dataset 

In [24]:
DF_mod.corr()

,air conditioner_5545,temperature,gen,temperature -1hr,temperature -2hr,temperature -3hr,temperature -4hr,temperature -5hr,temperature -6hr,hour,sin_hour,cos_hour,day_of_week,month,week_of_year,weekend,workingTime
air conditioner_5545,1.000000,0.567759,-0.010406,0.605930,0.633037,0.645768,0.643592,0.626838,0.596597,0.352808,-0.428578,0.207456,-0.014653,0.153875,0.164818,-0.006510,-0.051613
temperature,0.567759,1.000000,0.316497,0.991125,0.968990,0.936761,0.897785,0.855209,0.811763,0.238509,-0.327360,-0.141078,0.044240,0.217522,0.246750,0.041159,0.241258
gen,-0.010406,0.316497,1.000000,0.228523,0.132847,0.038199,-0.047481,-0.117825,-0.168933,0.164066,-0.231320,-0.740196,-0.028739,-0.026050,-0.018714,-0.028940,0.763218
temperature -1hr,0.605930,0.991125,0.228523,1.000000,0.991122,0.968982,0.936747,0.897763,0.855177,0.257253,-0.352895,-0.051384,0.043123,0.218107,0.247408,0.041167,0.162689
temperature -2hr,0.633037,0.968990,0.132847,0.991122,1.000000,0.991117,0.968975,0.936734,0.897741,0.263853,-0.354314,0.041905,0.042129,0.218714,0.248056,0.041090,0.076035
temperature -3hr,0.645768,0.936761,0.038199,0.968982,0.991117,1.000000,0.991111,0.968966,0.936714,0.255223,-0.331571,0.132361,0.041470,0.219407,0.248755,0.041314,-0.011671
temperature -4hr,0.643592,0.897785,-0.047481,0.936747,0.968975,0.991111,1.000000,0.991109,0.968959,0.226658,-0.286189,0.213863,0.041044,0.220099,0.249425,0.041808,-0.094104
temperature -5hr,0.626838,0.855209,-0.117825,0.897763,0.936734,0.968966,0.991109,1.000000,0.991107,0.172909,-0.221290,0.280817,0.040807,0.220860,0.250077,0.042359,-0.167530
temperature -6hr,0.596597,0.811763,-0.168933,0.855177,0.897741,0.936714,0.968959,0.991107,1.000000,0.099995,-0.141289,0.328665,0.040653,0.221641,0.250693,0.042820,-0.228670
hour,0.352808,0.238509,0.164066,0.257253,0.263853,0.255223,0.226658,0.172909,0.099995,1.000000,-0.776311,-0.102506,0.000388,-0.000966,-0.000344,0.000151,0.245312


Values near to 1 means high correlation wrt our consumption, principally the main correlations are: temperatures, hour and sin-hour